In [ ]:
from ocpy import OCP
from ocpy import UCRRSolver
from ocpy import symutils

import numpy as np
import sympy as sym
from sympy import sin, cos, tan, exp, log, ln, sinh, cosh, tanh, diff, sqrt
from IPython.display import display, Math


In [ ]:
n_x = 12
n_u = 6
sim_name = 'hexacopter'

ocp = OCP(n_x, n_u, sim_name)

t = ocp.get_t()
x = ocp.get_x()
u = ocp.get_u()


In [ ]:
m, l, k, Ixx, Iyy, Izz, gamma, g = ocp.define_scalar_constants(
    [('m', 1.44), ('l', 0.23), ('k', 1.6e-09),
     ('I_xx', 0.0348), ('I_yy', 0.0459), ('I_zz', 0.0977),
     ('gamma', 0.01), ('g', 9.80665)]
)
z_ref, u_min, u_max, epsilon = ocp.define_scalar_constants(
    [('z_ref', 5), ('u_min', 0.144), ('u_max', 6), ('epsilon', 0.01)]
)

q = ocp.define_vector_constant(
    'q',  [1, 1, 1, 0.01, 0.01, 0, 0.01, 0.01, 0.01, 0.1, 0.1, 0.001])
r = ocp.define_vector_constant(
    'r',  [0.01, 0.01, 0.01, 0.01, 0.01, 0.01])
q_f = ocp.define_vector_constant(
    'q_{f}',  [1, 1, 1, 0.01, 0.01, 0, 0.01, 0.01, 0.01, 0.1, 0.1, 0.001])
Q = symutils.diag(q)
R = symutils.diag(r)
Q_f = symutils.diag(q_f)


In [ ]:
p_ref = ocp.get_zero_vector(3)
p_ref[0] = sin(2*t)
p_ref[1] = 1 - cos(2*t)
p_ref[2] = z_ref + 2*sin(2*t)
# or directly
# p_ref = sym.Matrix([[sin(2*t)],
#                     [(1 - cos(2*t))],
#                     [z_ref + 2*sin(t)]])
p_ref_diff = p_ref.diff(t)

x_ref = ocp.get_zero_vector(n_x)
x_ref[0:3, :] = p_ref
x_ref[3:6, :] = p_ref_diff

U1 = sum(u[i] for i in range(n_u))
U2 = l*(-u[0]/2 - u[1] - u[2]/2 + u[3]/2 + u[4]+ u[5]/2)
U3 = l*(-(sqrt(3)/2)*u[0] + (sqrt(3)/2)*u[2] + (sqrt(3)/2)*u[3] - (sqrt(3)/2)*u[5])
U4 = k*(-u[0] + u[1] - u[2] + u[3] - u[4] + u[5]) - gamma * x[11]

f = ocp.get_zero_vector(n_x)
f[0] = x[6]
f[1] = x[7]
f[2] = x[8]
f[3] = x[9]
f[4] = x[10]
f[5] = x[11]
f[6] = (cos(x[5])*sin(x[4])*cos(x[3]) + sin(x[5])*sin(x[3]))*U1/m
f[7] = (sin(x[5])*sin(x[4])*cos(x[3]) - cos(x[5])*sin(x[3]))*U1/m
f[8] = -g + (cos(x[3])*cos(x[4]))*U1/m
f[9] = ((Iyy-Izz)/Ixx)*x[10]*x[11] + U2/Ixx
f[10] = ((Izz-Ixx)/Iyy)*x[9]*x[11] + U3/Iyy
f[11] = ((Ixx-Iyy)/Izz)*x[9]*x[10] + U4/Izz

u_ref = ocp.get_zero_vector(n_u)
for i in range(n_u):
    u_ref[i] = (m*g) / 6

u_barrier = epsilon * sym.Matrix([sum(-ln(u[i] - u_min) - ln(u_max - u[i]) for i in range(n_u))])
l = (x - x_ref).T * Q * (x - x_ref) + (u - u_ref).T * R * (u - u_ref) + u_barrier
lf = (x - x_ref).T * Q_f * (x - x_ref)

display(Math(r"\dot{x} = f(x, u, t) \equiv %s" % sym.latex(f)))
display(Math(r"l(x, u) = %s" % sym.latex(l)))
display(Math(r"l_f(x) = %s" % sym.latex(lf)))


In [ ]:
T = 5.0
N = 200

t0 = 0.0
x0 = np.array([0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0])

ocp.define_unconstrained(f, l, lf, t0, x0, T, N)


In [ ]:
solver = UCRRSolver(ocp)

us_guess = np.array([[1, 1, 1, 1, 1, 1] for i in range(N)])
xs_guess = np.tile(x0, (N + 1, 1))
lmds_guess = np.zeros((N + 1, n_x))

solver.set_guess(xs_guess=xs_guess, us_guess=us_guess, lmds_guess=lmds_guess)


In [ ]:
alphas = [0.5**i for i in range(8)]

# set hyperparameters.
solver.set_alphas(alphas)
solver.set_regularization_coeff(gamma_init=1e-3, rho_gamma=5.0, gamma_min=0.0, gamma_max=1e6)
solver.set_kkt_tol(kkt_tol=1e-3)
solver.set_max_iters(1000)

# Solve ocp
xs, us, ts, is_success = solver.solve(
    result=True, log=True, plot=True, gamma_fixed=0.0, enable_line_search=True
)


In [ ]:
xs

In [ ]:
%matplotlib inline
# Visualize
from ocpy.animator import HexacopterAnimator
animator = HexacopterAnimator(solver.get_log_directory(), sim_name)
animator.generate_animation(False)


In [ ]:
print('xs\n', xs)
print('us\n', us)


In [ ]:
result = solver.get_result()
gamma_hist = result['gamma_hist']

import matplotlib.pyplot as plt
import math

plt.plot(gamma_hist)
plt.title('gamma')
plt.show()

alpha_hist = result['alpha_hist']
plt.plot(alpha_hist)
plt.title('alpha')
plt.show()
print('average alpha:',sum(alpha_hist / (len(alpha_hist) - 1)))

cost_hist = result['cost_hist']
plt.plot(cost_hist)
plt.title('cost')
plt.show()

kkt_error_hist = result['kkt_error_hist']
plt.yscale('log')
plt.plot(kkt_error_hist)
plt.title('KKT error')
plt.show()

dyn_feas_hist = result['dyn_feas_hist']
dyn_feas_hist = np.where(dyn_feas_hist < 1e-20, np.nan, dyn_feas_hist)
plt.yscale('log')
plt.plot(kkt_error_hist, label='kkt_error')
plt.plot(dyn_feas_hist, label='dyn_feas')
plt.legend()
plt.title('dynamics feasibility error')
plt.show()
